In [7]:
!pip3 install catboost
!pip install category_encoders

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
# import visuals as vs
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15, 6.18*1.5]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier
import catboost
from sklearn.inspection import permutation_importance
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings
import sys
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.preprocessing import OrdinalEncoder

In [82]:
from google.colab import files

In [9]:
!gdown --id  1K7N4QCyCzPYoEBd3Mwvy6P5VGokT5su4  --output "train.csv" 
!gdown --id  1DpiEPo2WuNfCrDl7xfGhCWPsCKPNGRXm  --output "test.csv" 

Downloading...
From: https://drive.google.com/uc?id=1K7N4QCyCzPYoEBd3Mwvy6P5VGokT5su4
To: /content/train.csv
100% 116M/116M [00:02<00:00, 52.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DpiEPo2WuNfCrDl7xfGhCWPsCKPNGRXm
To: /content/test.csv
100% 116M/116M [00:00<00:00, 172MB/s]


In [41]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [53]:
train.head(5)

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,...,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,...,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,nan,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,nan,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,...,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,nan,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,...,0.763925,5.498902,3.423944,0.832518,7.375480,6.746988,6.942002,1.334611,18.256352,8.507281,2.503055,4.872157,2.573664,0.113967,12.554274,B,2.230754,1.990131,B,2.643678,J,nan,13.777666,10.574713,1.511063,4.949609,7.180722,5.655086e-01,1.166281,1.956521,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,NaN,6.320087,NaN,10.991098,NaN,NaN,NaN,NaN,NaN,NaN,6.414567,HIT,NaN,E,NaN,NaN,NaN,NaN,NaN,nan,A,NaN,NaN,6.083151,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,NaN,NaN,A,NaN,T,G,14.097099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [70]:
test.shape

(114393, 132)

In [44]:
# train[cat_col.columns] = train[cat_col.columns].astype(str)

In [54]:
X_train = train.drop(columns=['target'])
y_train = train['target']

In [56]:
cat_col = X_train.select_dtypes(include=[object])
cat_features = [X_train.columns.get_loc(col) for col in cat_col.columns]
cat_features

[3,
 22,
 24,
 30,
 31,
 47,
 52,
 56,
 66,
 71,
 74,
 75,
 79,
 91,
 107,
 110,
 112,
 113,
 125]

In [63]:
test[cat_col.columns] = test[cat_col.columns].astype(str)

In [47]:
encoder = ce.CatBoostEncoder(cols=cat_col.columns)
X_train_enc = encoder.fit_transform(X_train, y_train)
# X_test_enc = encoder.fit_transform(X_test, y_test)

In [57]:
params = {
          # "eval_metric": "TotalF1:average=Macro",
          'verbose': 200, 
          'random_seed': 72,
          'cat_features': cat_features,
          # 'nan_mode': 'Min',
          }
cat = CatBoostClassifier(**params)


cat.fit(X_train, y_train,
        use_best_model=True,
         )
print(cat.get_best_score())

You should provide test set for use best model. use_best_model parameter has been switched to false value.


Learning rate set to 0.077939
0:	learn: 0.6588561	total: 599ms	remaining: 9m 58s
200:	learn: 0.4464122	total: 1m 42s	remaining: 6m 47s
400:	learn: 0.4348665	total: 3m 22s	remaining: 5m 2s
600:	learn: 0.4255717	total: 5m 2s	remaining: 3m 20s
800:	learn: 0.4171900	total: 6m 42s	remaining: 1m 39s
999:	learn: 0.4092079	total: 8m 23s	remaining: 0us
{'learn': {'Logloss': 0.4092079078837474}}


In [86]:
preds_proba = cat.predict_proba(test)
# pred = cat.predict(X_test_enc)
df = pd.DataFrame(preds_proba, columns = ['2', 'PredictedProb'])

In [87]:
df['ID'] = test['ID']
df = df.drop(columns=['2'])


In [90]:
df = df[['ID', 'PredictedProb']]
df

,ID,PredictedProb
0,0,0.155152
1,1,0.908788
2,2,0.816667
3,7,0.677366
4,10,0.820594
...,...,...
114388,228700,0.885856
114389,228703,0.076752
114390,228704,0.857998
114391,228706,0.900448


In [89]:
df.to_csv('result.csv', index=False)

--------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# X = train.drop(columns=['target'])
# y = train['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
# X_train.shape

(80024, 132)

In [ ]:
encoder = ce.CatBoostEncoder(cols=cat_features.columns)
X_train_enc = encoder.fit_transform(X_train, y_train)
X_test_enc = encoder.fit_transform(X_test, y_test)

In [ ]:
X_train_enc

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,...,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
29464,59013,0.880353,7.185261,0.761247,3.797819,9.773418,2.365737,2.671651,0.208154,8.663367,1.312911,15.332371,6.786415,4.048835,10.991098,2.015568,6.378379,2.213597,0.787335,0.242784,16.708896,6.174171,0.761247,1.194494e-07,0.761247,0.342598,1.844550,1.978245,6.288218,8.150917,0.761247,0.761247,1.784500,2.515297,7.625762,8.484024,14.251156,0.356234,0,0.194224,...,0.491465,5.184050,4.824091,0.574077,6.403807,7.175572,7.635528,1.595286,18.743268,6.876926,1.415716,7.138836,2.993672,0.147252,9.775812,0.761247,2.251954,4.655909,0.761247,4.609110,0.761247,0.761247,15.373629,12.216217,2.351961,11.570503,7.531806,3.602042,0.789945,2.840096,5.742574,3.076092,0.091608,0.761247,1.875899,1.389361,2.129615,0,1.882951,0.756757
102185,204266,1.675050,4.057894,0.880623,3.800061,8.431594,2.273642,2.615695,2.393458,9.727274,5.754923,16.086486,8.245033,4.299372,13.920420,1.901042,5.600000,2.128557,0.674959,0.222378,17.346160,7.708944,0.761247,1.205166e+01,0.761247,4.722011,1.346413,2.545271,4.248449,8.048289,0.761247,0.761247,1.614687,2.515090,8.979138,8.692154,14.899322,0.692309,0,0.113281,...,0.496481,5.442994,4.326065,0.580927,6.659960,5.000000,7.691203,1.936620,5.738221,4.424102,3.778443,5.536266,2.412525,1.524369,9.128581,0.761247,2.146905,3.972963,0.761247,5.010060,0.761247,0.880623,15.467055,13.280000,1.624999,9.401629,5.615385,1.741687,0.900524,2.855465,6.636364,6.305384,1.091426,0.761247,2.065684,4.017858,1.790215,1,1.923077,1.440000
62783,125308,0.760274,4.827218,0.920416,4.270565,7.646269,3.027398,2.253424,0.153458,8.881789,1.466084,15.408320,6.676628,3.306412,12.793943,1.586768,5.934066,2.533935,1.660895,0.244997,16.179701,7.792484,0.761247,9.188857e-07,0.761247,0.144685,1.996575,2.993150,6.241180,7.643836,0.880623,0.880623,1.428082,2.493150,7.345749,8.931506,15.288519,0.364742,0,9.335971,...,0.557204,4.250058,4.105980,0.654568,6.013698,9.300913,6.659988,0.816210,10.714894,6.143718,3.680963,6.207095,2.336829,0.092885,9.668418,0.880623,2.520020,3.223685,0.880623,4.602740,0.761247,0.761247,15.177218,12.307693,2.741936,3.971632,9.665654,1.380736,1.404256,2.997544,5.431311,2.983812,0.031903,0.761247,1.540304,2.940924,1.845946,0,2.212767,0.659340
73600,146772,NaN,NaN,0.690312,NaN,NaN,NaN,NaN,NaN,NaN,0.787746,NaN,6.558603,NaN,12.512040,NaN,NaN,NaN,NaN,NaN,NaN,8.183679,0.761247,NaN,0.880623,NaN,NaN,NaN,NaN,NaN,0.761247,0.587082,NaN,NaN,5.950799,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.761247,NaN,NaN,0.587082,NaN,0.761247,0.761247,13.682760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.761247,NaN,NaN,NaN,0,NaN,NaN
83080,165868,NaN,NaN,0.552249,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,NaN,6.369190,NaN,11.272973,NaN,NaN,NaN,NaN,NaN,NaN,7.015470,0.761247,NaN,0.587082,NaN,NaN,NaN,NaN,NaN,0.380623,0.440312,NaN,NaN,11.090442,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587082,NaN,NaN,0.440312,NaN,0.761247,0.761247,18.751232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.761247,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76820,153340,NaN,NaN,0.759533,NaN,NaN,NaN,NaN,NaN,NaN,0.634574,NaN,6.158370,NaN,10.115593,NaN,NaN,NaN,NaN,NaN,NaN,5.960320,0.705568,NaN,0.762839,NaN,NaN,NaN,NaN,NaN,0.725180,0.720924,NaN,NaN,7.684999,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.756496,NaN,NaN,0.833065,NaN,0.772638,0.8254

In [ ]:
cat_features = list(range(X.shape[1]))
cat_features_names = cat_col.columns # here we specify names of categorical features
cat_features = [cat_col.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [ ]:
params = {
          # "eval_metric": "TotalF1:average=Macro",
          'verbose': 200, 
          'random_seed': 72,
          # 'cat_features': cat_features.columns.to_list(),
          # 'nan_mode': 'Min',
          }
cat = CatBoostClassifier(**params)


cat.fit(X_train_enc, y_train,
        use_best_model=True,
         )
print(cat.get_best_score())

You should provide test set for use best model. use_best_model parameter has been switched to false value.


Learning rate set to 0.066928
0:	learn: 0.6620634	total: 95.2ms	remaining: 1m 35s
200:	learn: 0.4567734	total: 16.2s	remaining: 1m 4s
400:	learn: 0.4368052	total: 31.8s	remaining: 47.5s
600:	learn: 0.4203277	total: 47.2s	remaining: 31.3s
800:	learn: 0.4056648	total: 1m 2s	remaining: 15.6s
999:	learn: 0.3922251	total: 1m 18s	remaining: 0us
{'learn': {'Logloss': 0.3922250814989878}}


In [ ]:
preds_proba = cat.predict_proba(X_test_enc)
# pred = cat.predict(X_test_enc)
log_loss(y_test, preds_proba)

0.4713018529056414

In [ ]:
imputer = KNNImputer(n_neighbors=1).fit_transform(X_train, y_train)
enc = OrdinalEncoder(cat_features.columns.to_list()).fit_transform(X_train, y_train)
# pipeline = Pipeline(steps=[('i', imputer)('enc', enc)])


ValueError: ignored

In [ ]:
Classifier = RandomForestClassifier(random_state=(72))
param_grid = {
    # 'bootstrap': [True],
    # 'max_depth': [15, 16, 17, 18, 19, 20, None],
    # 'max_features': ['auto', 1, 2, 3, 4]
    # 'min_samples_leaf': [1, 2, 3],
    # 'min_samples_split': [2, 5, 10, 15, 100],
    # 'n_estimators': [200, 300, 400]
}

grid = GridSearchCV(estimator = Classifier, param_grid = param_grid, cv = 10, n_jobs = -1, scoring='f1_macro')
grid.fit(X_train_k, y_train)

NameError: ignored